In [ ]:
import numpy as np
import pandas as pd
#import folium
import math 
import os
import time

In [ ]:
df1 = pd.read_csv("CaliforniaRoadNetwork_Edges.csv")
df2 = pd.read_csv("CaliforniaRoadNetwork_Nodes.csv")


In [ ]:
df1

In [ ]:
df2

In [ ]:
# import folium
# m = folium.Map(
#     location=[41.974556, -121.904167],
#     zoom_start=7.5
    
# )
# for i in range(len(df2)):
#         m.add_child(
#         folium.CircleMarker(
#               [df2["Latitude"][i],df2["Longitude"][i]],
#              radius=2, # define how big you want the circle markers to be
#             color='yellow',
#             fill=True,
#             fill_color='red',
#              fill_opacity=1.5
#      )
#     )
# m.add_child(folium.LatLngPopup())
# display(m)
# m.save('Position Visualization.html')

In [ ]:
def get_EuclideanDistance(node1,node2): 
    node1_Longitude = float(df2.loc[df2["NodeID"]==node1, :]["Longitude"])
    node1_Latitude = float(df2.loc[df2["NodeID"]==node1, :]["Latitude"])
   
    node2_Longitude = float(df2.loc[df2["NodeID"]==node2, :]["Longitude"])
    node2_Latitude = float(df2.loc[df2["NodeID"]==node2, :]["Latitude"])

    return math.sqrt(pow(node1_Longitude-node2_Longitude,2) + pow(node1_Latitude-node2_Latitude,2))

In [ ]:
def get_Dict():
    graph_search = {}
    ##进入循环
    #单向算一次
    for i in range(len(df2)):
        next_node = []
        this_node = int(df2.loc[i,["NodeID"]])                             #遍历所有ID
        endnode = df1.loc[df1["StartNodeID"]==this_node, :]["EndNodeID"]   #找到这个id对应所有的终点ID
        for e in endnode:
            #groupby 找到所有的node 然后遍历每个node
            # if e not in next_node:
            child_node = (e,get_EuclideanDistance(this_node,e))
            next_node.append(child_node)
        graph_search[this_node] = next_node
    
    #反向算一次
    for i in range(len(df2)):
        this_node = int(df2.loc[i,["NodeID"]])                             #遍历所有ID
        start_node = df1.loc[df1["EndNodeID"]==this_node, :]["StartNodeID"]   #找到这个id对应所有的终点ID
        if len(start_node) != 0:
            for e in start_node:
                child_node = (e,get_EuclideanDistance(this_node,e))
                graph_search[this_node].append(child_node)
    return graph_search

In [ ]:
graph_search = get_Dict()

In [ ]:
# graph_search_pd = pd.DataFrame.from_dict(graph_search, orient='index')
# graph_search_pd

In [ ]:
def sld_to_des(des_node):
    graph_search_sld = {}
    for i in range(len(df2)):
        graph_search_sld[i] = get_EuclideanDistance(int(des_node),i)
    return graph_search_sld

In [ ]:
print(sld_to_des(5))

In [ ]:
def bfs(graph_to_search, initial_state, goal_state, verbose=False):
    # this is a list of a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in this list represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost.
    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []

    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
        
        path = frontiers.pop(0)  # Get the first element in the list
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.
            
            # check if we reached the goal state or not
            if next_node == goal_state:
                goal_path = new_path[1:]
                goal_cost = new_path[0]
                return goal_path, goal_cost  # if yes, we can return this path and its cost
            else:
                frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)

    return None

In [ ]:
len(graph_search)

In [ ]:
graph_search

# Q1
![avator](./200_Question/question1.png)

In [ ]:
print(bfs(graph_search, 0, 1894, verbose=False))
print(bfs(graph_search, 4, 3115, verbose=False))
print(bfs(graph_search, 18, 9186, verbose=False))
print(bfs(graph_search, 25, 15061, verbose=False))
print(bfs(graph_search, 33, 21040, verbose=False))


In [ ]:
print(bfs(graph_search, 0, 1, verbose=False))


# Q2
![avator](./200_Question/question2.png)

### 2.1 BFS

### 2.2 DFS

### 2.3 UniformSearch

### 2.4 GreedySearch

In [ ]:
def greedy_search(graph_to_search, initial_state, goal_state, heuristics_function, verbose=False):
    # this is a list of a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in this list represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost.
    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []

    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
            
        # get the nodes in frontiers to be expanded
        frontier_heuristics = []
        for x in frontiers:
            frontier_heuristics.append(heuristics_function[x[-1]])
        idx_to_pop = frontier_heuristics.index(min(frontier_heuristics))
        
        path = frontiers.pop(idx_to_pop)
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.
            
            # check if we reached the goal state or not
            if next_node == goal_state:
                goal_path = new_path[1:]
                goal_cost = new_path[0]
                return goal_path, goal_cost  # if yes, we can return this path and its cost
            else:
                frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)

    return None

In [ ]:
for i in range(1000):
    initial_state = np.random.randint(0,21047)
    goal_state = np.random.randint(0,21047)
    print('The',i+1,'th combinations')
    print(greedy_search(graph_search, initial_state, goal_state, heuristics_function=sld_to_des(goal_state), verbose=False))
    print('----------')

### 2.5 A* Search

In [ ]:
def a_star_search(graph_to_search, initial_state, goal_state, heuristics_function, verbose=False):
    # this is a list of a list because we need to eventually return
    # the entire PATH from the initial state to the goal state. So,
    # each element in this list represents a path from the the initial
    # state to one frontier node. We use the first element in each path
    # to represent the cost.
    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []

    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
            
        # get the nodes in frontiers to be expanded
        estimated_path_cost = []
        for x in frontiers:
            heuristic_value = heuristics_function[x[-1]]
            path_cost = x[0]
            estimated_path_cost.append(heuristic_value+path_cost)
        idx_to_pop = estimated_path_cost.index(min(estimated_path_cost))
        
        path = frontiers.pop(idx_to_pop)
        node = path[-1]  # Get the last node in this path
        
        if node == goal_state:
            goal_path = path[1:]
            goal_cost = path[0]
            return goal_path, goal_cost
        
        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.

            frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)

    return None

In [ ]:
for i in range(1000):
    initial_state = np.random.randint(0,21047)
    goal_state = np.random.randint(0,21047)
    print('The',i+1,'th combinations')
    print(a_star_search(graph_search, initial_state, goal_state, heuristics_function=sld_to_des(goal_state), verbose=False))
    print('----------')

# Q3
![avator](./200_Question/question3.png)

In [ ]:
# pip uninstall requests

In [ ]:
# pip uninstall memory-profiler

In [ ]:
pip install resource

In [8]:
import time
import resource 

time_start = time.perf_counter()

for i in range(1000):
    initial_state = np.random.randint(0,21047)
    goal_state = np.random.randint(0,21047)
    print('The',i+1,'th combinations')
    print(a_star_search(graph_search, initial_state, goal_state, heuristics_function=sld_to_des(goal_state), verbose=False))
    print('----------')

time_elapsed = (time.perf_counter() - time_start)
memMb=resource.getrusage(resource.RUSAGE_SELF).ru_maxrss/1024.0/1024.0
print (time_elapsed,'secs',memMb,'MByte')

ModuleNotFoundError: No module named 'resource'

# Q4
![avator](./200_Question/question4.png)

### Depth Tree

In [ ]:
### Depth TREE
def dfs_tree(graph_to_search, initial_state, goal_state, verbose=False):
    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []
    ver = 0 # to
    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
        path = frontiers.pop(-1)  # Get the last element in the list
        node = path[-1]  # Get the last node in this path
        
        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        
        if len(actions) == 0:#If next actions is empty, then pass this node,discard visited nodes
            visited = copy.deepcopy(frontiers[-1])# Cover the old visited with the frontiers
            del visited[0]# delate the cost information
            del visited[-1]# delate the unvisited node
            continue
            
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.
            
            # check if we reached the goal state or not
            if next_node == goal_state:
                goal_path = new_path[1:]
                goal_cost = new_path[0]
                return goal_path, goal_cost  # if yes, we can return this path and its cost
            else:
                frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)
        ver += 1
    return None

In [ ]:
for i in range(100):
    start_node = np.random.randint(0,len(df2))
    end_node = np.random.randint(0,len(df2))
    print('---',dfs_tree(graph_search,start_node,end_node,verbose = False),'\n\n\n')


### Depth limited 

In [ ]:
def Depth_Limited_Search(graph_to_search, initial_state, goal_state, limited_deepth,verbose=False):

    frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
    visited = []
    
    while len(frontiers) > 0:   # use while loop to iteratively perform search
        if verbose:  # print out detailed information in each iteration
            print('Frontiers (paths):')
            for x in frontiers:
                print('  -', x)
            print('Visited:', visited)
            print('\n')
        
        path = frontiers.pop(-1)  # Get the last element in the list
        node = path[-1]  # Get the last node in this path

        if len(path)-1 == limited_deepth: 
            continue

        if node in visited:  # check if we have expanded this node, if yes then skip this
            continue
            
        actions = graph_to_search[node] # get the possible actions
        for next_node, next_cost in actions:
            new_path = path.copy()
            new_path.append(next_node)
            new_path[0] = new_path[0] + next_cost
            
            if next_node in visited or new_path in frontiers:
                continue  # skip this node if it is already in the frontiers or the visited list.
            
            # check if we reached the goal state or not
            if next_node == goal_state:
                goal_path = new_path[1:]
                goal_cost = new_path[0]
                return goal_path, goal_cost  # if yes, we can return this path and its cost
            else:
                frontiers.append(new_path)  # add to the frontiers
        
        # after exploring all actions, we add this node to the visited list
        visited.append(node)
    return None

In [ ]:
Depth_Limited_Search(graph_search,0,1894,200,verbose = False)

### Iterative_Deepening_dfs

In [ ]:
def iterative_Deepening_dfs(graph_to_search, initial_state, goal_state, verbose=False):
    limited_deepth = 0
    now_deepth = -1

    while now_deepth < limited_deepth:
        frontiers = [[0, initial_state]]  # the frontier list only has the initial state, with a cost of 0.
        visited = []
        now_deepth += 1
        
        while len(frontiers) > 0:   # use while loop to iteratively perform search
            if verbose:  # print out detailed information in each iteration
                print('Frontiers (paths):')
                for x in frontiers:
                    print('  -', x)
                print('Visited:', visited)
                print('\n')
            
            path = frontiers.pop(-1)  # Get the last element in the list
            node = path[-1]  # Get the last node in this path

            if len(path)-1 == limited_deepth: 
                continue

            if node in visited:  # check if we have expanded this node, if yes then skip this
                continue
                
            actions = graph_to_search[node] # get the possible actions
            for next_node, next_cost in actions:
                new_path = path.copy()
                new_path.append(next_node)
                new_path[0] = new_path[0] + next_cost
                
                if next_node in visited or new_path in frontiers:
                    continue  # skip this node if it is already in the frontiers or the visited list.
                
                # check if we reached the goal state or not
                if next_node == goal_state:
                    goal_path = new_path[1:]
                    goal_cost = new_path[0]
                    return goal_path, goal_cost  # if yes, we can return this path and its cost
                else:
                    frontiers.append(new_path)  # add to the frontiers
            
            # after exploring all actions, we add this node to the visited list
            visited.append(node)
        
        limited_deepth += 1
        now_deepth = -1
    return None

In [ ]:
iterative_Deepening_dfs(graph_search,0,1894,verbose = False)

In [ ]:
for i in range(100):
    start_node = np.random.randint(0,len(df2))
    end_node = np.random.randint(0,len(df2))
    print(iterative_Deepening_dfs(graph_search,start_node,end_node,verbose = False))


In [ ]:
def iterative_deepening_search_test(graph_to_search, initial_state, goal_state, verbose=False):
    limit = 0
    while True:
        frontiers = [[0, initial_state]]
        visited = []

        while len(frontiers) > 0:
            if verbose:
                print("Frontiers (paths):", frontiers)
                print("Visited:", visited, "\n")

            path = frontiers.pop(-1)
            node = path[-1]

            if len(path)-1 == limit:
                continue

            if node in visited:
                continue

            actions = graph_to_search[node]
            for next_node, next_cost in actions:
                new_path = path.copy()
                new_path.append(next_node)
                new_path[0] = new_path[0] + next_cost

                if next_node in visited or new_path in frontiers:
                    continue

                if next_node == goal_state:
                    goal_path = new_path[1:]
                    goal_cost = new_path[0]
                    return goal_path, goal_cost
                else:
                    frontiers.append(new_path)

            visited.append(node)
        limit += 1

In [ ]:

for i in range(100):
    start_node = np.random.randint(0,len(df2))
    end_node = np.random.randint(0,len(df2))
    print(iterative_deepening_search_test(graph_search,start_node,end_node,verbose = False))
